In [61]:
import queue, pygame, time, copy
import math, pickle

In [24]:
time_start = time.time()
BLACK = (0, 0, 0)
WHITE = (200, 200, 200)
GREEN = (0, 255, 0,)
BLUE = (0, 0, 255)
RED = (255, 0, 0)
YELLOW = (255 ,255 ,0)
WINDOW_HEIGHT = 850
WINDOW_WIDTH = 850

WIDTH = 3.3
HEIGHT = WIDTH
MARGIN = 5
FPS = 30

pygame.init()
pygame.mixer.init()

SCREEN = pygame.display.set_mode((WINDOW_WIDTH, WINDOW_HEIGHT))
pygame.display.set_caption("Python Maze Generator")
clock = pygame.time.Clock()
SCREEN.fill(BLACK)

def drawGrid(a):

    for row in range(len(a)):
        for column in range(len(a[0])):
            color = WHITE
            if a[row][column] == '*':
                color = RED
            if a[row][column] == 'S':
                color = BLUE
            if a[row][column] == 'T':
                color = YELLOW
            pygame.draw.rect(SCREEN,
                             color,
                             [(MARGIN + WIDTH) * column + MARGIN,
                              (MARGIN + HEIGHT) * row + MARGIN,
                              WIDTH,
                              HEIGHT])
    pygame.display.update()

In [25]:
# function to calculate the manhattan distance

def manhattanDistance(start, goal):
    return abs(start[0] - goal[0]) + abs(start[1] - goal[1])

def getNeighbours(grid_dim, pos):
    neighbours = []
    
    if pos[0] > 0:
        neighbours.append((pos[0] - 1, pos[1]))
    if pos[0] < grid_dim[0]:
        neighbours.append((pos[0], pos[1] + 1))
    if pos[1] > 0:
        neighbours.append((pos[0], pos[1] - 1))
    if pos[1] < grid_dim[1]:
        neighbours.append((pos[0] + 1, pos[1]))
    
    return neighbours

In [119]:

# Function to implement A* search algorithm


class aStar:
    def __init__(self, grid, known, start, goal, seen):
        """
            Function that calculates the shortest path from start to goal given a grid and the known world
            known contains h value for each cell
            If a cell is an obstacle, the h value becomes infinity, 

            Args:
                grid (list): grid 
                known (list): known grid
                start (tuple): start coordinates
                goal (tuple): goal coordinates
        """    
        self.grid_dim = (len(grid[0]), len(grid))
        self.openList = queue.PriorityQueue()   
        self.closedList = []
        self.known = known
        self.grid = grid
        self.openList.put((0, start, []))
        self.path = []
        self.seen = seen
        # self.block = block
        # g Values
        

    def run(self):    
        gMatrix = [[0 for i in range(len(self.grid[0]))] for j in range(len(self.grid))]
        while self.openList.empty() == False:
        # for i in range(20):
            # print("Open List is", self.openList.queue)
            # print("Closed List is", self.closedList)
            _, current, history = self.openList.get()
            # print("Agent is at", current)
            neighbours = getNeighbours(self.grid_dim, current)
            self.seen.extend(neighbours)
            # Observe surroundings
            for i in range(len(neighbours)):
                # print(neighbours[i])
                if neighbours[i] == goal:
                    # print("Found goal")
                    #TODO: Return path
                    history.extend([current, goal])
                    self.path = history
                    return sorted(set(self.path), key=self.path.index)
                    # return self.path
                
                # # update obstacle values
                # if self.grid[neighbours[i][0]][neighbours[i][1]] == "*":
                #     self.known[neighbours[i][0]][neighbours[i][1]] = math.inf
                g = gMatrix[current[0]][current[1]] + 1
                h = known[neighbours[i][0]][neighbours[i][1]]
                f = g + h
                if f == math.inf:
                    continue
                present, f_check = self.checkCellInOpenList(neighbours[i])
                # print("Checking open list")
                if present:
                    if f >= f_check:
                        continue
                
                present, f_check = self.checkCellInClosedList(neighbours[i])
                # print("Checking closed list")
                if present:
                    # print(neighbours[i], "present in closed list", f, f_check)
                    if f >= f_check:
                        # print("Adding ", neighbours[i], "with f =", f_check, f,"to open list")
                        continue    
                # print("Adding ", neighbours[i], "with f =", f,"to open list")
                history.append(current)
                self.openList.put((f, neighbours[i], history))
                gMatrix[neighbours[i][0]][neighbours[i][1]] = g    
                
            self.closedList.append((gMatrix[current[0]][current[1]], current))

    
    def checkCellInOpenList(self, pos):
        for i in self.openList.queue:
            if i[1] == pos:
                return True, i[0]
        return False, 0
    
    def checkCellInClosedList(self, pos):
        for i in self.closedList:
            if i[1] == pos:
                return True, i[0]
        return False, 0
    
    
    

In [192]:
class RepeatedAstar:
    
    def __init__(self, grid, start, goal):
        self.grid = grid
        self.start = start
        self.current = start
        self.goal = goal
        self.known = [[manhattanDistance((j, i), goal) for i in range(len(grid[0]))] for j in range(len(grid))]
        # self.path = [start]
        self.currAstarPath = None
        self.path = []
        self.new_grid = copy.deepcopy(grid)
        self.seen = []
        self.block = []
        self.final_path = []
    
    def runAstar(self):

        neighbours = getNeighbours((len(self.grid), len(self.grid[0])), self.current)
        print(neighbours)
        for i in range(len(neighbours)):    
            if self.grid[neighbours[i][0]][neighbours[i][1]] == "*":
                self.known[neighbours[i][0]][neighbours[i][1]] = math.inf
                self.block.append(neighbours[i])
            
        while self.current != self.goal:
            print("Agent is at", self.current)
            self.path.append(self.current)
            # print(known)
            pathFinder = aStar(self.grid, self.known, self.current, self.goal, self.seen)
            self.seen = pathFinder.seen
            self.currAstarPath = pathFinder.run()
            self.current = self.currAstarPath[1]
            self.display()
            
        print("Reached goal")
        
    def display(self):
        if len(self.currAstarPath) == 0:
            return
        drawGrid(self.grid)
        li = self.currAstarPath
        print(li)
        for i in li:
            pygame.draw.rect(SCREEN,
                            GREEN,
                            [(MARGIN + WIDTH) * i[1] + MARGIN + MARGIN-2,
                            (MARGIN + HEIGHT) * i[0] + MARGIN + MARGIN-2,
                            WIDTH-8,
                            HEIGHT-8])
        pygame.display.update()
        ci = 0
        f = self.new_grid
        #print(li)
        for i in li:
            x = i[0]
            y = i[1]
            #print(x," ",y)
            if(f[x][y]) != "*":
                f[x][y] = "\033[1;32;43mS"
                tu = (x,y)
                if(ci > 0):
                    f[prevx][prevy] = "\033[1;32;43m "
                prevx = x
                prevy = y
            else:
                break
            ci = ci+1

        countx = 0
        seen = self.seen
        block = self.block
        final_path = self.final_path
        print(seen)
        for e in f:
            county = 0
            for r in e:
                if((countx, county) in block or (countx, county) in seen):
                    print("\033[1;34;40m",r,end="")

                else:
                    print("\033[0;37;41m",r,end="")
                county = county+1
            print("\033[0;37;40m\n",end='')
            countx = countx+1
        print(li)
        for i in li:
            pygame.draw.rect(SCREEN,
                                BLUE,
                                [(MARGIN + WIDTH) * i[1] + MARGIN + MARGIN-2,
                                (MARGIN + HEIGHT) * i[0] + MARGIN + MARGIN-2,
                                WIDTH-8,
                                HEIGHT-8])
            
            pygame.display.update()
            #   time.sleep(0.1)
            if(i[0] == tu[0] and i[1] == tu[1]):
                break
            final_path.append(i)
            
        a[self.current[0]][self.current[1]] = " "
        a[tu[0]][tu[1]] = "S"
        self.new_grid = f
        self.final_path = final_path
        return tu
    

In [193]:
# a = [
#     ['*', '*', '*', '*', '*', '*', '*', '*', '*', '*'],
#     ['*', 'S', '*', '*', '*', ' ', ' ', ' ', ' ', '*'],
#     ['*', ' ', '*', '*', '*', ' ', '*', ' ', ' ', '*'],
#     ['*', ' ', '*', '*', '*', ' ', '*', '*', ' ', '*'],
#     ['*', ' ', '*', '*', ' ', ' ', '*', '*', ' ', '*'],
#     ['*', ' ', '*', '*', ' ', ' ', '*', '*', ' ', '*'],
#     ['*', ' ', ' ', ' ', ' ', '*', '*', '*', ' ', '*'],
#     ['*', '*', '*', ' ', ' ', '*', '*', '*', 'T', '*'],
#     ['*', '*', '*', '*', '*', '*', '*', '*', '*', '*']
# ]
# start = (1, 1)
# goal = (7, 8)

mazes = pickle.load(open('mazes.pkl', 'rb'))
a, start, goal = mazes[0]

known = [[manhattanDistance((j, i), goal) for i in range(len(a[0]))] for j in range(len(a))]
# known

In [194]:
start, goal

((74, 63), (23, 31))

In [195]:
# # Test A star
# known = [[manhattanDistance((j, i), goal) for i in range(len(a[0]))] for j in range(len(a))]
# test = aStar(a, known, start, goal)
# test.run()

In [196]:
x = RepeatedAstar(a, start, goal)
x.runAstar()

[(73, 63), (74, 64), (74, 62), (75, 63)]
Agent is at (74, 63)
[(74, 63), (73, 63), (72, 63), (71, 63), (70, 63), (69, 63), (68, 63), (67, 63), (66, 63), (65, 63), (64, 63), (63, 63), (62, 63), (61, 63), (60, 63), (59, 63), (58, 63), (57, 63), (56, 63), (55, 63), (54, 63), (53, 63), (52, 63), (51, 63), (50, 63), (49, 63), (48, 63), (47, 63), (46, 63), (45, 63), (44, 63), (43, 63), (42, 63), (41, 63), (40, 63), (39, 63), (38, 63), (37, 63), (36, 63), (35, 63), (34, 63), (33, 63), (32, 63), (31, 63), (30, 63), (29, 63), (28, 63), (27, 63), (26, 63), (25, 63), (24, 63), (23, 63), (23, 62), (23, 61), (23, 60), (23, 59), (23, 58), (23, 57), (23, 56), (23, 55), (23, 54), (23, 53), (23, 52), (23, 51), (23, 50), (23, 49), (23, 48), (23, 47), (23, 46), (23, 45), (23, 44), (23, 43), (23, 42), (23, 41), (23, 40), (23, 39), (23, 38), (23, 37), (23, 36), (23, 35), (23, 34), (23, 33), (23, 32), (23, 31)]
[(73, 63), (74, 64), (74, 62), (75, 63), (72, 63), (73, 64), (73, 62), (74, 63), (71, 63), (72, 6

In [186]:
x.path

[(74, 63),
 (74, 63),
 (73, 63),
 (72, 63),
 (71, 63),
 (70, 63),
 (69, 63),
 (68, 63),
 (67, 63),
 (66, 63),
 (65, 63),
 (64, 63),
 (63, 63),
 (62, 63),
 (61, 63),
 (60, 63),
 (59, 63),
 (58, 63),
 (57, 63),
 (56, 63),
 (55, 63),
 (54, 63),
 (53, 63),
 (52, 63),
 (51, 63),
 (50, 63),
 (49, 63),
 (48, 63),
 (47, 63),
 (46, 63),
 (45, 63),
 (44, 63),
 (43, 63),
 (42, 63),
 (41, 63),
 (40, 63),
 (39, 63),
 (38, 63),
 (37, 63),
 (36, 63),
 (35, 63),
 (34, 63),
 (33, 63),
 (32, 63),
 (31, 63),
 (30, 63),
 (29, 63),
 (28, 63),
 (27, 63),
 (26, 63),
 (25, 63),
 (24, 63),
 (23, 63),
 (23, 62),
 (23, 61),
 (23, 60),
 (23, 59),
 (23, 58),
 (23, 57),
 (23, 56),
 (23, 55),
 (23, 54),
 (23, 53),
 (23, 52),
 (23, 51),
 (23, 50),
 (23, 49),
 (23, 48),
 (23, 47),
 (23, 46),
 (23, 45),
 (23, 44),
 (23, 43),
 (23, 42),
 (23, 41),
 (23, 40),
 (23, 39),
 (23, 38),
 (23, 37),
 (23, 36),
 (23, 35),
 (23, 34),
 (23, 33),
 (23, 32)]

In [43]:
x.path

AttributeError: 'RepeatedAstar' object has no attribute 'path'

In [9]:
neighbours = [(5, 8), (6, 9), (6, 7), (7, 8)]

In [10]:
for i in range(len(neighbours)):
    print(x.known[neighbours[i][0]][neighbours[i][1]])

2
2
2
0


In [41]:
# for i in a:
#     for j in i:
#         print(j, end="")
#     print("")